In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from PIL import Image, ImageTk
from tkinter import *
import matplotlib.pyplot as plt


In [2]:
import os
import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
def load_img(filename):
  img = open(filename,'rb')
  img = img.read()
  img = tf.image.decode_jpeg(img)
  img = tf.image.resize(img, [200, 256])
  img /= 255.0
  return img

In [4]:
def create1input(img):
  img = np.array(img)
  img = img.reshape((-1,200,256,3))
  return img

In [5]:
def create_truth_mask(filename):
  img = open(filename,'rb')
  img = img.read()
  img = tf.image.decode_jpeg(img,channels=3)
  img = tf.image.resize(img, [200, 256],method = 'nearest')
  img = np.array(img)
  img = img.reshape((200,256,3))
  return img

In [6]:
class_names20 = ['road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic_light',
'traffic_sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car', 'truck', 'bus',
'train', 'motorcycle' , 'bicycle']

In [7]:
def get_fn(target_img,pred_img,i):
    tar = target_img == i
    pre = pred_img==i
    ans = (tar|pre) & ~pre
    return sum(sum(ans))
def get_fp(target_img,pred_img,i):
    tar = target_img == i
    pre = pred_img==i
    ans = (tar|pre) & ~tar
    return sum(sum(ans))
def get_tp(target_img,pred_img,i):
    tar = target_img == i
    pre = pred_img==i
    ans = tar&pre
    return sum(sum(ans))

def get_iou(target_img,pred_img,NUM_CLASS=20):
    iou_classes=[]
    class_names=[]
    for i in range(20):
        if(i in np.unique(target_img) and i in np.unique(pred_img)):
            den = get_tp(target_img,pred_img,i)+get_fp(target_img,pred_img,i)+get_fn(target_img,pred_img,i)
            if(int(den) != 0 and get_tp(target_img,pred_img,i) != 0):
                if(i==0):
                    class_names.append("Unlabelled")
                else:
                    class_names.append(class_names20[i-1])
                t = get_tp(target_img,pred_img,i)/(get_tp(target_img,pred_img,i)+get_fp(target_img,pred_img,i)+get_fn(target_img,pred_img,i))
                
                iou_classes.append(round(t , 3))
    return iou_classes , class_names

# def get_detailed_iou(target_img,pred_img,NUM_CLASS = 20):
#     iou_classes = []
#     for i in range(20):
#         den = get_tp(target_img,pred_img,i)+get_fp(target_img,pred_img,i)+get_fn(target_img,pred_img,i)
#         if(den != 0):
#             iou_classes.append(get_tp(target_img,pred_img,i)/(get_tp(target_img,pred_img,i)+get_fp(target_img,pred_img,i)+get_fn(target_img,pred_img,i)))
#         else:
#             iou_classes.append(0)
#     return iou_classes

def present_iou(iou_classes , class_names):
    iou = ""
    d = dict(zip(class_names , iou_classes))
    d = sorted(d.items() , key=lambda x:x[1])
    
    for i in range(len(iou_classes) - 1 , -1,-1):
        iou += ( "\n" + d[i][0] + " : "+str(d[i][1]) + "\t" +"\n")
            
    return iou

In [8]:
colormap20=[[128, 64, 128], [244, 35, 232], [70, 70, 70], [102, 102, 156], 
                      [190, 153, 153], [153, 153, 153], [250, 170, 30], [220, 220, 0], [107, 142, 35], 
                      [152, 251, 152], [70, 130, 180], [220, 20, 60], [255, 0, 0], [0, 0, 142], [0, 0, 70],
                      [0, 60, 100], [0, 80, 100], [0, 0, 230], [119, 11, 32] ]
a=np.zeros(19)

def one_hot_encoded_mask(img):
    # seg = load_and_preprocess_mask(path)
    output_mask=[]
    for i,color in enumerate(colormap20):
        cmap = np.all(np.equal(img,color),axis=-1)
        output_mask.append(cmap*1)
    mask = np.array(output_mask)
    mask = np.transpose(mask,(1,2,0))
    left_map = np.all(np.equal(mask,a),axis=-1)
    output_mask.insert(0,left_map)
    output_mask = np.stack(output_mask,axis=-1)
    return output_mask

def calculate_iou(model,t_img,p_img):
  p = model.predict(p_img)
  pred_img = p[0]
  pred_img = np.argmax(pred_img,axis=-1)
  ti = one_hot_encoded_mask(t_img)
  target_img = np.argmax(ti,axis=-1)

  _p = LayersToRGBImage(p[0])
  _s = LayersToRGBImage(ti)
  print("Mean IoU score : ",MeanIoU(target_img,pred_img))

In [9]:
def LayersToRGBImage(img):
    colors =[[0,0,0],[128, 64, 128], [244, 35, 232], [70, 70, 70], [102, 102, 156], 
                      [190, 153, 153], [153, 153, 153], [250, 170, 30], [220, 220, 0], [107, 142, 35], 
                      [152, 251, 152], [70, 130, 180], [220, 20, 60], [255, 0, 0], [0, 0, 142], [0, 0, 70],
                      [0, 60, 100], [0, 80, 100], [0, 0, 230], [119, 11, 32] ]
    
    nimg = np.zeros((img.shape[0], img.shape[1], 3))
    for i in range(img.shape[2]):
        c = img[:,:,i]
        col = colors[i]
        
        for j in range(3):
            nimg[:,:,j]+=col[j]*c
    nimg = nimg/255.0
    return nimg

In [10]:
def show_predictions(model,imgs):
  p = model.predict(imgs)
  for i in range(p.shape[0]):
    _p = LayersToRGBImage(p[i])
    predimg = _p

    plt.figure(figsize=(15,5))
    plt.subplot(131)
    plt.title("Actual Image")
    plt.imshow(imgs[i])

    plt.subplot(132)
    plt.title("Predicted Image")
    plt.imshow(_p)
    # plt.tight_layout()

    plt.show()

In [11]:
i2m = keras.models.load_model("C:\\Users\\HAPPY\\OneDrive\\Desktop\\PRISM\\unetnirdin50epochs.h5")

In [12]:
from tkinter import ttk

In [13]:
from PIL import Image

In [14]:
import time

In [15]:
win = tk.Tk()
win.geometry("1200x1000")
win['background'] = '#b3b3b3'

font = ('MS Sans Serif', 24, 'bold')
title = Label(win, text='IIITDM - SRIB (Semantic Segmentation)                                                                                                              ')
title.config(bg='#404040', fg='#4d79ff')
title.config(font=font)
title.config(height=3, width=120)
title.place(x=15, y=5)

font1 = ('times', 12, 'bold')
text = Text(win, height=20, width=80)

# Input Old Image
frame = Frame(win, width=300, height=300, bg="#e6e6e6", colormap="new")
# frame.pack()
frame.place(x=50, y=120)

font = ('black', 10, 'bold')
Imagetext = Label(win, text='Original Image' , relief=SUNKEN)
Imagetext.config(bg='white', fg='#4e03fc')
Imagetext.config(font=font)
Imagetext.config(height=3, width=20)
Imagetext.place(x=120, y=450)

# Output Old Image
frame1 = Frame(win, width=300, height=300, bg="#e6e6e6", colormap="new")
# frame1.pack()
frame1.place(x=500, y=120)

font = ('black', 10, 'bold')
Imagetext = Label(win, text='Predicted Mask' , relief=SUNKEN)
Imagetext.config(bg='white', fg='#4e03fc')
Imagetext.config(font=font)
Imagetext.config(height=3, width=20)
Imagetext.place(x=560, y=450)

#Classwise IoU
frame2 = Frame(win, width=300, height=300, bg="white", colormap="new")
# frame2.grid()
frame2.place(x=930, y=120)

font = ('black', 10, 'bold')
IoUtext = Label(win, text='Classwise IoU' , relief=SUNKEN)
IoUtext.config(bg='white', fg='#4e03fc')
IoUtext.config(font=font)
IoUtext.config(height=3, width=20)
IoUtext.place(x=990, y=450)

#IIIT logo
frame3 = Frame(win, width=100, height=100, bg="white", colormap="new")
frame3.grid()
frame3.place(x=20, y=10)

img = Image.open("iiitk_logo.png")
img = img.resize((100, 100))  # Resize the image as needed
img_tk = ImageTk.PhotoImage(img)

shoe_label = tk.Label(frame3, image=img_tk)
shoe_label.pack()

#Samsung Logo
frame4 = Frame(win, width=100, height=100, bg="white", colormap="new")
frame4.grid()
frame4.place(x=1150, y=10)
img_logo = Image.open("prism-logo.png")
img_logo = img_logo.resize((100, 100))  # Resize the image as needed
img_tk_logo = ImageTk.PhotoImage(img_logo)
sho_label = tk.Label(frame4, image=img_tk_logo)
sho_label.pack()

from PIL import Image
from torch.autograd import Variable
import matplotlib.image as mpimg



def show_prediction(image):
    img = np.array(image)
    img = tf.image.resize(img, [256, 256])
    img /= 255.0

    img = np.array(img)
    img = img.reshape((-1 , 256 , 256 , 3))
    p = i2m.predict(img)
    
    pred_img = p[0]
    p = LayersToRGBImage(p[0])
    p = (p * 255).astype(np.uint8)
    p = Image.fromarray(p)
    p.save("image.png")
    
    mask = Image.open("image.png")
    img_resized=mask.resize((300,300)) # new width & height
    p=ImageTk.PhotoImage(img_resized)
    label1 = Label(frame1, image = p)
    label1.pack() 
    
    return pred_img
    

def show_iou(pred_img , tar_img):
#     img = np.array(image)
#     img = tf.image.resize(img, [256, 256])
#     img /= 255.0

#     img = np.array(img)
#     img = img.reshape((-1 , 256 , 256 , 3))
#     p = i2m.predict(img)
    
#     pred_img = p[0]
#     p = LayersToRGBImage(p[0])
#     p = (p * 255).astype(np.uint8)
#     p = Image.fromarray(p)
#     p.save("image.png")
    
#     mask = Image.open("image.png")
#     img_resized=mask.resize((300,300)) # new width & height
#     p=ImageTk.PhotoImage(img_resized)
#     label1 = Label(frame1, image = p)
#     label1.pack()    
    
    t_img = np.array(tar_img)
    t_img = t_img[:, :, :3]
    t_img = tf.image.resize(t_img, [256, 256])
    t_img = np.array(t_img)
    
    ti = one_hot_encoded_mask(t_img)
    
    pred_img = np.argmax(pred_img,axis=-1)
    target_img = np.argmax(ti,axis=-1)
    
    print("Target_img - " , target_img.shape)
    print("Pred_Img - " , pred_img.shape)
    
    iou_classes , class_names = get_iou(target_img , pred_img)
    class_iou = present_iou(iou_classes ,class_names)
    print(class_iou)
    
    font = ('black', 10, 'bold')
    textIou = tk.Label(frame2, text=class_iou)
    textIou.config(font=font)
    textIou.pack()
    

def upload_file():
    global img
    for widget in frame.winfo_children():
        widget.destroy()
    f_types = [("PNG Files", "*.png")]
    filename = filedialog.askopenfilename(filetypes=f_types)
    img=Image.open(filename)
    image2= img
    img_resized=img.resize((300,300)) # new width & height
    img=ImageTk.PhotoImage(img_resized)
    label = Label(frame, image = img)
    label.pack()

    pred_img = show_prediction(image2)
    
    
    filename_tar = filedialog.askopenfilename(filetypes=f_types)
    tar_img=Image.open(filename_tar)
#     time.sleep(5)
    show_iou(pred_img , tar_img)
    
# Define a function to clear the selected frame
def clear_frame():
    for widget in frame.winfo_children():
        widget.destroy()
    
    
    for widget in frame1.winfo_children():
        widget.destroy()
        
    for widget in frame2.winfo_children():
        widget.destroy()

 
def close():
   win.destroy()

b1 = tk.Button(win,text='Upload \nPhoto', width=20,command = lambda:upload_file() , relief=RAISED)
b1.config(font=('times', 12, 'bold'))
b1.place(x=170, y=570)

# b2 = tk.Button(win,text='Upload \nMask', width=20,command = lambda:upload_mask())
# b2.config(font=('times', 12, 'bold'))
# b2.place(x=310, y=570)

b2 = tk.Button(win, text="Clear",width = 20 , height = 2, command=lambda: clear_frame())
b2.config(font=('times', 12, 'bold'))
b2.place(x = 500 , y = 570)


exitButton = Button(win, text="Exit", height = 2 , width = 10, command=close ,relief=RAISED)
exitButton.place(x=900, y=570)
exitButton.config(font=('times', 12, 'bold'))

win.mainloop() 

1/1 [==============================] - 8s 8s/step
Target_img -  (256, 256)
Pred_Img -  (256, 256)

road : 0.955	

sky : 0.879	

vegetation : 0.872	

building : 0.866	

Unlabelled : 0.769	

sidewalk : 0.723	

car : 0.343	

person : 0.147	

traffic_sign : 0.099	

pole : 0.088	



In [67]:
# from PIL import Image
# import numpy as np

# # Create a numpy array with shape (100, 100, 3)
# arr = np.zeros((100, 100, 3), dtype=np.uint8)

# # Convert the numpy array to a PIL image
# img = Image.fromarray(arr)

# # Save the image
# img.save("image.png")

In [32]:
# i2m.summary()